In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import time
import arducam_mipicamera as arducam
import json
import cv2
import numpy as np

In [2]:
# Camera settings
def align_down(size, align):
    return(size & ~((align) -1))
def align_up(size, align):
    return align_down(size + align - 1, align)

def get_frame(camera):
    frame = camera.capture(encoding = "i420")
    fmt = camera.get_format()
    height = int(align_up(fmt['height'], 16))
    width = int(align_up(fmt['width'], 32))
    image = frame.as_array.reshape(int(height * 1.5), width)
    image = cv2.cvtColor(image, cv2.COLOR_YUV2BGR_I420)
    image = image[: fmt['height'], :fmt['width']]
    return image

In [3]:
try:
    camera_params = json.load(open('camera_params.txt', 'r'))
except Exception as e:
    print(e)
    print('Missing Camera Parameters please Find camera parameiter')

In [4]:
print(camera_params)

{'height': 480, 'mode': 10, 'width': 1280}


In [5]:
# set up camera
camera = arducam.mipi_camera()
print('Opeing Camera')
camera.init_camera()
print('Setting Mode')
mode = camera_params['mode']
camera.set_mode(mode)
print('Getting format')
fmt = camera.get_format()
print('Current Mode:{}, Resolution: {}x{}'.format(fmt['mode'], fmt['width'], fmt['height']))


Opeing Camera
Setting Mode
Getting format
Current Mode:10, Resolution: 1280x480


In [6]:
# Camera Settings
cam_width = camera_params['width']
cam_height = camera_params['height']

# Buffer for capture image settings
img_width = camera_params['width']
img_height = camera_params['height']
print('Scaled image rsolution:{}x{}'.format(str(img_width), str(img_height)))

Scaled image rsolution:1280x480


In [7]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [8]:
# set up needs resrouces
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [9]:
# test camera
while True:
    frame = get_frame(camera)
    frame = cv2.resize(frame, (img_width, img_height))
    
    # face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Draw stuff
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    cv2.imshow('Camera', frame)
    key = cv2.waitKey(1) & 0xFF
    
    if(key == ord('q')):
        break
# Break out of while loop
cv2.destroyAllWindows()    